In [2]:
import os

In [3]:
%pwd

'/home/sabeer/Desktop/Projects/Chicken-Disease-CNN-Classifier/research'

In [4]:
os.chdir('../')

In [5]:
%pwd

'/home/sabeer/Desktop/Projects/Chicken-Disease-CNN-Classifier'

In [6]:
#update the entity
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [7]:
#update configuration manager
from chickenCNN.constants import *
from chickenCNN.utils.common import read_yaml, create_directories

In [8]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
    
    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            (model_ckpt_dir),
            (config.tensorboard_root_log_dir)
        ])

        prepare_callback_config = PrepareCallbacksConfig(
            root_dir=(config.root_dir),
            tensorboard_root_log_dir=(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=(config.checkpoint_model_filepath)
        )

        return prepare_callback_config    

In [9]:
#Update the components
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

2024-06-26 12:36:02.838663: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-26 12:36:08.943692: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-26 12:36:09.003028: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-26 12:36:22.670251: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [10]:
class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config
    
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
        )


    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]

In [11]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()
    
except Exception as e:
    raise e

[2024-06-26 12:44:26,080: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-06-26 12:44:26,231: INFO: common: yaml file: params.yaml loaded successfully]
[2024-06-26 12:44:26,281: INFO: common: created directory at: artifacts]
[2024-06-26 12:44:26,336: INFO: common: created directory at: artifacts/prepare_callbacks/checkpoint_dir]
[2024-06-26 12:44:26,338: INFO: common: created directory at: artifacts/prepare_callbacks/tensorboard_log_dir]


In [13]:
import time
timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
f"tb_logs_at_{timestamp}"

'tb_logs_at_2024-06-26-12-45-52'